In [40]:
import pandas as pd
import numpy as np
from datetime import datetime
import openpyxl
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
import warnings
warnings.filterwarnings('ignore')

In [41]:
# создаем таблицу – данные о продажах
sales_data = {
    'ID_клиента': ['001', '001', '002', '003', '003', '004'],
    'Дата_покупки': ['01.03.2025', '01.06.2025', '05.03.2025', '10.03.2025', '15.05.2025', '12.03.2025'],
    'Сумма': [10000, 3333, 10000, 10000, 3333, 10000],
    'Продукт': ['Годовой курс', 'Продление', 'Годовой курс', 'Годовой курс', 'Продление', 'Годовой курс']
}

df_sales = pd.DataFrame(sales_data)
df_sales['Дата_покупки'] = pd.to_datetime(df_sales['Дата_покупки'], format='%d.%m.%Y')

In [42]:
df_sales

,ID_клиента,Дата_покупки,Сумма,Продукт
0,001,2025-03-01,10000,Годовой курс
1,001,2025-06-01,3333,Продление
2,002,2025-03-05,10000,Годовой курс
3,003,2025-03-10,10000,Годовой курс
4,003,2025-05-15,3333,Продление
5,004,2025-03-12,10000,Годовой курс


In [43]:
# создаем таблицу – данные о посещаемости
attendance_data = {
    'ID_студента': ['001', '002', '003', '004'],
    'Неделя_1': [1, 1, 1, 0],
    'Неделя_2': [1, 0, 1, 1],
    'Неделя_3': [1, 0, 1, 0],
    'Неделя_4': [1, 0, 0, 1]
}

df_attendance = pd.DataFrame(attendance_data)

In [44]:
df_attendance

,ID_студента,Неделя_1,Неделя_2,Неделя_3,Неделя_4
0,001,1,1,1,1
1,002,1,0,0,0
2,003,1,1,1,0
3,004,0,1,0,1


## Расчёты

In [45]:
# Расчёты для продаж
total_clients = df_sales['ID_клиента'].nunique()
clients_purchase_count = df_sales['ID_клиента'].value_counts()
repeat_clients = (clients_purchase_count > 1).sum()
retention_rate = (repeat_clients / total_clients) * 100

# Определение первой покупки
first_purchases = df_sales.groupby('ID_клиента')['Дата_покупки'].min().reset_index()
first_purchases.columns = ['ID_клиента', 'Первая_покупка']
df_with_first = df_sales.merge(first_purchases, on='ID_клиента')
df_with_first['Тип_покупки'] = df_with_first.apply(
    lambda row: 'Новая' if row['Дата_покупки'] == row['Первая_покупка'] else 'Повторная', 
    axis=1
)

avg_new_purchase = df_with_first[df_with_first['Тип_покупки'] == 'Новая']['Сумма'].mean()
avg_repeat_purchase = df_with_first[df_with_first['Тип_покупки'] == 'Повторная']['Сумма'].mean()

# Расчёты для посещаемости
weeks_cols = ['Неделя_1', 'Неделя_2', 'Неделя_3', 'Неделя_4']
weekly_attendance = df_attendance[weeks_cols].mean()
df_attendance['Общие_посещения'] = df_attendance[weeks_cols].sum(axis=1)
engaged_students = df_attendance[df_attendance['Общие_посещения'] >= 3]
num_engaged = len(engaged_students)

In [46]:
# создаем Excel файл
wb = openpyxl.Workbook()
wb.remove(wb.active)  # Удаляем стандартный лист

# Стили для оформления
header_font = Font(bold=True, size=12, color="FFFFFF")
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
title_font = Font(bold=True, size=14, color="2F4F4F")
metric_font = Font(bold=True, size=11, color="1F4E79")
border = Border(left=Side(style='thin'), right=Side(style='thin'), 
               top=Side(style='thin'), bottom=Side(style='thin'))
center_alignment = Alignment(horizontal='center', vertical='center')

In [47]:
# Лист – анализ продаж
ws1 = wb.create_sheet(title="Анализ продаж")

# Заголовок листа
ws1['A1'] = "АНАЛИЗ ПРОДАЖ И ПОВТОРНЫХ ПОКУПОК"
ws1['A1'].font = Font(bold=True, size=16, color="2F4F4F")
ws1.merge_cells('A1:D1')
ws1['A1'].alignment = center_alignment

# Дата создания отчёта
ws1['A2'] = f"Дата создания отчёта: {datetime.now().strftime('%d.%m.%Y %H:%M')}"
ws1['A2'].font = Font(italic=True, size=10)
ws1.merge_cells('A2:D2')

# Таблица с данными о продажах
ws1['A4'] = "Данные о продажах:"
ws1['A4'].font = title_font

# Заголовки таблицы
headers = ['ID клиента', 'Дата покупки', 'Сумма', 'Продукт']
for col, header in enumerate(headers, 1):
    cell = ws1.cell(row=5, column=col, value=header)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = center_alignment
    cell.border = border

# Данные таблицы
df_sales_display = df_sales.copy()
df_sales_display['Дата_покупки'] = df_sales_display['Дата_покупки'].dt.strftime('%d.%m.%Y')

for row, (_, data) in enumerate(df_sales_display.iterrows(), 6):
    for col, value in enumerate(data, 1):
        cell = ws1.cell(row=row, column=col, value=value)
        cell.border = border
        cell.alignment = center_alignment

# Расчёты под таблицей
calc_row = 6 + len(df_sales_display) + 2

ws1[f'A{calc_row}'] = "РЕЗУЛЬТАТЫ АНАЛИЗА:"
ws1[f'A{calc_row}'].font = title_font

calculations = [
    ("Общее число клиентов:", total_clients),
    ("Количество клиентов с повторной покупкой:", repeat_clients),
    ("Retention Rate:", f"{retention_rate:.1f}%"),
    ("Средний чек новой покупки:", f"{avg_new_purchase:.0f} руб."),
    ("Средний чек повторной покупки:", f"{avg_repeat_purchase:.0f} руб.")
]

for i, (metric, value) in enumerate(calculations, 1):
    ws1[f'A{calc_row + i}'] = metric
    ws1[f'A{calc_row + i}'].font = metric_font
    ws1[f'B{calc_row + i}'] = value
    ws1[f'B{calc_row + i}'].font = Font(size=11)

# Выводы и рекомендации
conclusions_row = calc_row + len(calculations) + 2
ws1[f'A{conclusions_row}'] = "ВЫВОДЫ И РЕКОМЕНДАЦИИ:"
ws1[f'A{conclusions_row}'].font = title_font

recommendations = [
    "• Retention Rate составляет 50% - хороший показатель для образовательных услуг",
    "• Средний чек повторной покупки значительно ниже первой (3333 vs 10000 руб.)",
    "• Рекомендуется развивать программы лояльности для повышения повторных продаж",
    "• Стоит проанализировать причины снижения суммы при повторных покупках"
]

for i, rec in enumerate(recommendations, 1):
    ws1[f'A{conclusions_row + i}'] = rec
    ws1[f'A{conclusions_row + i}'].font = Font(size=10)

# Настройка ширины колонок
ws1.column_dimensions['A'].width = 25
ws1.column_dimensions['B'].width = 20
ws1.column_dimensions['C'].width = 15
ws1.column_dimensions['D'].width = 20

In [48]:
# Лист – анализ посещаемости
ws2 = wb.create_sheet(title="Анализ посещаемости")

# Заголовок листа
ws2['A1'] = "АНАЛИЗ ВОВЛЕЧЁННОСТИ СТУДЕНТОВ"
ws2['A1'].font = Font(bold=True, size=16, color="2F4F4F")
ws2.merge_cells('A1:E1')
ws2['A1'].alignment = center_alignment

# Дата создания отчёта
ws2['A2'] = f"Дата создания отчёта: {datetime.now().strftime('%d.%m.%Y %H:%M')}"
ws2['A2'].font = Font(italic=True, size=10)
ws2.merge_cells('A2:E2')

# Таблица с данными о посещаемости
ws2['A4'] = "Данные о посещаемости:"
ws2['A4'].font = title_font

# Заголовки таблицы
attendance_headers = ['ID студента', 'Неделя 1', 'Неделя 2', 'Неделя 3', 'Неделя 4']
for col, header in enumerate(attendance_headers, 1):
    cell = ws2.cell(row=5, column=col, value=header)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = center_alignment
    cell.border = border

# Данные таблицы
for row, (_, data) in enumerate(df_attendance[['ID_студента'] + weeks_cols].iterrows(), 6):
    for col, value in enumerate(data, 1):
        cell = ws2.cell(row=row, column=col, value=value)
        cell.border = border
        cell.alignment = center_alignment

# Расчёты под таблицей
calc_row = 6 + len(df_attendance) + 2

ws2[f'A{calc_row}'] = "РЕЗУЛЬТАТЫ АНАЛИЗА:"
ws2[f'A{calc_row}'].font = title_font

# Средняя посещаемость по неделям
ws2[f'A{calc_row + 1}'] = "Средняя посещаемость по неделям:"
ws2[f'A{calc_row + 1}'].font = metric_font

week_metrics = []
for i, (week, attendance) in enumerate(weekly_attendance.items(), 2):
    week_display = f"Неделя {week.split('_')[1]}"
    ws2[f'A{calc_row + i}'] = f"  {week_display}:"
    ws2[f'B{calc_row + i}'] = f"{attendance:.1%}"
    week_metrics.append((week_display, f"{attendance:.1%}"))

# Дополнительные метрики
additional_calc_row = calc_row + len(weekly_attendance) + 3
additional_metrics = [
    ("Общее количество студентов:", len(df_attendance)),
    ("Вовлечённые студенты (3+ посещений):", f"{num_engaged} из {len(df_attendance)}"),
    ("Процент вовлечённых студентов:", f"{(num_engaged/len(df_attendance)*100):.1f}%")
]

for i, (metric, value) in enumerate(additional_metrics):
    ws2[f'A{additional_calc_row + i}'] = metric
    ws2[f'A{additional_calc_row + i}'].font = metric_font
    ws2[f'B{additional_calc_row + i}'] = value
    ws2[f'B{additional_calc_row + i}'].font = Font(size=11)

# Детальная таблица по студентам
detail_row = additional_calc_row + len(additional_metrics) + 2
ws2[f'A{detail_row}'] = "ДЕТАЛЬНАЯ ИНФОРМАЦИЯ ПО СТУДЕНТАМ:"
ws2[f'A{detail_row}'].font = title_font

# Заголовки детальной таблицы
detail_headers = ['ID студента', 'Общие посещения', 'Статус']
for col, header in enumerate(detail_headers, 1):
    cell = ws2.cell(row=detail_row + 1, column=col, value=header)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = center_alignment
    cell.border = border

# Данные детальной таблицы
for row, (_, data) in enumerate(df_attendance.iterrows(), detail_row + 2):
    student_id = data['ID_студента']
    total_visits = data['Общие_посещения']
    status = "Вовлечён" if total_visits >= 3 else "Требует внимания"
    
    ws2.cell(row=row, column=1, value=student_id).border = border
    ws2.cell(row=row, column=2, value=total_visits).border = border
    status_cell = ws2.cell(row=row, column=3, value=status)
    status_cell.border = border
    
    # Цветовое выделение статуса
    if status == "Вовлечён":
        status_cell.fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")
    else:
        status_cell.fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")

# Рекомендации
recommendations_row = detail_row + len(df_attendance) + 3
ws2[f'A{recommendations_row}'] = "РЕКОМЕНДАЦИИ:"
ws2[f'A{recommendations_row}'].font = title_font

student_recommendations = [
    "• Посещаемость снижается к концу курса (с 75% до 50%)",
    "• 50% студентов являются вовлечёнными (3+ посещений)",
    "• Необходима работа со студентами 002 и 004 (низкая посещаемость)",
    "• Рекомендуется анализ причин снижения активности на 2-3 неделях",
    "• Стоит внедрить систему напоминаний и мотивации"
]

for i, rec in enumerate(student_recommendations, 1):
    ws2[f'A{recommendations_row + i}'] = rec
    ws2[f'A{recommendations_row + i}'].font = Font(size=10)

# Настройка ширины колонок
ws2.column_dimensions['A'].width = 25
ws2.column_dimensions['B'].width = 20
ws2.column_dimensions['C'].width = 15
ws2.column_dimensions['D'].width = 15
ws2.column_dimensions['E'].width = 15

In [49]:
# сохраняем файл
filename = f"analytics_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
wb.save(filename)